In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
# 데이터 로드
odf = pd.read_pickle('./dataset/20_basic_feature_df.pkl')
df = odf.copy()
odf

,TRD_NAME_2,TRD_COUNTRY_2,TRD_ADDR_2,CUS_TOTAL_NUMBER_OF_ITEMS_5,CUS_TOTAL_NUMBER_OF_PACKAGES_6,CUS_REF_NO_7,CON_TIN_8,CON_NAME_8,CON_COUNTRY_8,CON_ADDR_8,...,CAL_ADDITIONAL_RATE_OF_TAX_47,PAM_FINANCIAL_VALUE_47,CAL_METHOD_OF_PAYMENT_47,IMP_DATE_OF_DECLARATION_54,REP_TIN_54,PERSON_NAME_54,GEND_REFERENCE_54,GEND_ISSUE_DATE_54,ACCEPTANCE_DATE,LABEL
0,ООО ВИП-ТРАНС,RU,НИЖНЕВАРТОВСК КУЗОВАТКИНА ДОМ 5 СТРОЕНИЕ 3,1,64.0,762235/140217/0000189,620010597,ООО ЧИТГАР,TJ,ИСТАРАВШАН БУСТОН 107,...,SP,553.76,БН,20170214,510000091,ВАХОБОВ ШАВКАТ ТУРСУНБОЕВИЧ,СГ01/007,20150624.0,"14.02.17 11:55:18,551000000",0
1,ООО ВИП-ТРАНС,RU,НИЖНЕВАРТОВСК КУЗОВАТКИНА ДОМ 5 СТРОЕНИЕ 3,1,64.0,762235/140217/0000189,620010597,ООО ЧИТГАР,TJ,ИСТАРАВШАН БУСТОН 107,...,AD,17286.89,УН,20170214,510000091,ВАХОБОВ ШАВКАТ ТУРСУНБОЕВИЧ,СГ01/007,20150624.0,"14.02.17 11:55:18,551000000",0
2,ООО ВИП-ТРАНС,RU,НИЖНЕВАРТОВСК КУЗОВАТКИНА ДОМ 5 СТРОЕНИЕ 3,1,64.0,762235/140217/0000189,620010597,ООО ЧИТГАР,TJ,ИСТАРАВШАН БУСТОН 107,...,AD,62232.82,БН,20170214,510000091,ВАХОБОВ ШАВКАТ ТУРСУНБОЕВИЧ,СГ01/007,20150624.0,"14.02.17 11:55:18,551000000",0
3,""" М/С АКИДЖ ДУТЕ МИЛЛС ЛТД"",Ч/З БАНДАР АБАС П/...",BD,АББАС,1,1213.0,762241/140217/0100665,560002910,"ООО ""РОСТОИ СУГД""",TJ,ГУЛИСТОН И.СОМОНИ 1,...,SP,553.76,БН,20170214,510000091,САЙФИЕВА МАВЗУНА ШАРИФДЖАНОВНА,СГ/01/0077,20150624.0,"14.02.17 15:53:44,921000000",0
4,""" М/С АКИДЖ ДУТЕ МИЛЛС ЛТД"",Ч/З БАНДАР АБАС П/...",BD,АББАС,1,1213.0,762241/140217/0100665,560002910,"ООО ""РОСТОИ СУГД""",TJ,ГУЛИСТОН И.СОМОНИ 1,...,AD,8662.33,УН,20170214,510000091,САЙФИЕВА МАВЗУНА ШАРИФДЖАНОВНА,СГ/01/0077,20150624.0,"14.02.17 15:53:44,921000000",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146503,"ЗАО ""КЕМА""",LT,МАРИЯМПОЛЕ,8,8.0,762243/251120/0002915,640007700,"ЧДММ ""ОРЗУИ КОРВОН""",TJ,ИСТИКЛОЛ Б.ГАФУРОВ МАХАЛАИ 1,...,AD,2732.34,БН,20201125,510020624,ДЖУРАЕВ ИЛХОМДЖОН ИБРАГИМОВИЧ,СГ/01/0107,20181210.0,"25.11.20 11:39:55,465000000",0
2146504,"ЗАО ""КЕМА""",LT,МАРИЯМПОЛЕ,8,8.0,762243/251120/0002915,640007700,"ЧДММ ""ОРЗУИ КОРВОН""",TJ,ИСТИКЛОЛ Б.ГАФУРОВ МАХАЛАИ 1,...,SP,4486.05,БН,20201125,510020624,ДЖУРАЕВ ИЛХОМДЖОН ИБРАГИМОВИЧ,СГ/01/0107,20181210.0,"25.11.20 11:39:55,465000000",0
2146505,"ЗАО ""КЕМА""",LT,МАРИЯМПОЛЕ,8,8.0,762243/251120/0002915,640007700,"ЧДММ ""ОРЗУИ КОРВОН""",TJ,ИСТИКЛОЛ Б.ГАФУРОВ МАХАЛАИ 1,...,AD,6217.52,БН,20201125,510020624,ДЖУРАЕВ ИЛХОМДЖОН ИБРАГИМОВИЧ,СГ/01/0107,20181210.0,"25.11.20 11:39:55,465000000",0
2146506,XINJING AKBATA INTERNATIONAL FREIGHT FORWARDER...,CN,УРУМЧИ,14,2998.0,762513/251120/0017517,70016467,ООО МАДИНА-2015,TJ,ТУРСУНЗОДА ТУРСУНЗАДЕ 136,...,AD,63.00,КТ,20201125,010006305,ГУЛБАЕВА МУКАДДАС ХАЗРАТКУЛОВНА,ДБ/03/0115,20200729.0,"25.11.20 16:22:43,918000000",0


# 기본 파생변수 생성

In [5]:
# 적발내역 데이터 로드
ill_df = pd.read_pickle('./dataset/01_detection_detail.pkl')

# 아래에서 볼 수 있듯이 모든 적발건이 데이터셋 존재하지 않으므로 샘플별로 각각 적발건을 계산 하는 것이 아닌,
# 적발 내역 데이터에 적힌 수치 (NUMB_OF_DETECTED)를 각 해당업체 / 원산지 / HS CODE / 관세사 번호에 일괄적으로 부여
cnt = 0

for i in range(len(ill_df)) :
  cust_number = ill_df.iloc[i, 4]
  hs_code = ill_df.iloc[i, 5]
  if len(df[(df.CUS_REF_NO_7 == cust_number) & (df.COM_COMBINED_NOMENCLATURE_33 == hs_code)]) != 0 :
    cnt += 1
  else : pass

print('전체 적발 건 403개중 {}개만 데이터 셋에 존재'.format(cnt))

전체 적발 건 403개중 295개만 데이터 셋에 존재


In [6]:
# 파생변수 생성을 위해 CUS_REF_NO_7 기준으로 DATE 칼럼 생성

# 편집할 DataFrame 생성
cus_date_df = pd.DataFrame()
cus_date_df['CUS_REF_NO_7'] = df['CUS_REF_NO_7']

# CUS_REF_NO_7 의 날짜 부분을 편집하여 DATE 칼럼 생성
cus_date_df['DATE'] = cus_date_df.apply(lambda x : '20' + x['CUS_REF_NO_7'][11:13] +
                                        x['CUS_REF_NO_7'][9:11] + x['CUS_REF_NO_7'][7:9], axis=1)

# df에도 DATE 칼럼 삽입 (0번째), datetime으로 type 변경
df.insert(0, 'DATE', cus_date_df['DATE'])
df['DATE'] = df['DATE'].astype('datetime64')

## 해당업체 (CON_NAME_8 기준)

- 최근 6개월간 수입건수 : CUS_REF_NO_7 기준

- 최근 6개월간 수입금액 : VAL_FINANCIAL_VALUE_12 기준

- 최근 6개월간 검사건수 : 2020. 05. 26 - 2020. 11. 25 까지 LABEL 1인 row

- 최근 6개월간 적발건수 : 적발내역 엑셀 참조

- 최근 6개월간 적발율 : 적발건수 / 검사건수

In [ ]:
# 아래서 볼 수 있듯이 VAL_FINANCIAL_VALUE_12는 신고서 하나에 한개씩 존재
# 이를 파생변수 산출에 고려 

tmp_df = df[['CUS_REF_NO_7', 'VAL_FINANCIAL_VALUE_12']]
aggregations = {'VAL_FINANCIAL_VALUE_12' : lambda x : len(x.unique())}
tmp_df_grouped = tmp_df.groupby('CUS_REF_NO_7').agg(aggregations)

cnt = 0
for value in tmp_df_grouped.VAL_FINANCIAL_VALUE_12 :
  if value != 1. :
    cnt += 1
  else : pass

print('신고서 번호당 하나 이상의 VAL_FINANCIAL_VALUE를 갖고 있는 경우 : {}'.format(cnt))

신고서 번호당 하나 이상의 VAL_FINANCIAL_VALUE를 갖고 있는 경우 : 0


In [7]:
# 파생변수 생성 사용자 함수 작성

def get_dev_con(df) :

  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  import pickle

  target_date = df['DATE'][2146407]
  delta = target_date - relativedelta(months=6)
  tdf = df[df.DATE >= delta]
  tdf.reset_index(drop=True, inplace=True)

  ##### 수입건수, 수입금액 산출 #####

  edf = tdf[['CON_NAME_8', 'CUS_REF_NO_7', 'VAL_FINANCIAL_VALUE_12', 'LABEL']]

  # 수입건수는 CUS_REF_NO_7의 유니크값 수
  aggregations1 = {'CUS_REF_NO_7' : lambda x : len(x.unique())}
  edf_grouped1 = edf.groupby('CON_NAME_8').agg(aggregations1)
  # 해당 업체 이름 : 수입 건수 dict 작성 / pickle로 저장
  count_dict = dict(zip(edf_grouped1.index, edf_grouped1.CUS_REF_NO_7))
  with open('./dataset/Feature_Engineering_dict/R6M_CON_IMP_CNT_dict.pkl', 'wb') as f_cnt :
    pickle.dump(count_dict, f_cnt)
  
  # 수입 금액은 해당 업체가 신고한 신고서 하나 당 VAL_FINANCIAL_VALUE_12의 합계
  # 집계함수로 최빈값을 추출 ; 신고서 하나당 VAL_FINANCIAL_VALUE_12는 한개이므로 
  aggregations2 = {'VAL_FINANCIAL_VALUE_12' : lambda x : x.mode()}
  edf_grouped2 = edf.groupby(['CON_NAME_8', 'CUS_REF_NO_7']).agg(aggregations2)
  # 2중 인덱스이므로 0번째 값 (CON_NAME_8) 만 리스트에 담음
  idx_list = []
  for i in range(len(edf_grouped2.index)) :
    idx_list.append(edf_grouped2.index[i][0])
  # 2중 인덱스를 CON_NAME_8 만으로 바꾼 DataFrame 생성 
  amt_df = pd.DataFrame({'CON_NAME_8' : idx_list,
                         'VAL_FINANCIAL_VALUE_12' : edf_grouped2.VAL_FINANCIAL_VALUE_12.values})
  # 집계함수 산출
  aggregations3 = {'VAL_FINANCIAL_VALUE_12' : 'sum'}
  amt_df_grouped = amt_df.groupby('CON_NAME_8').agg(aggregations3)
  # 해당 업체 이름 : 수입 금액 dict 작성 / pickle로 저장
  amt_dict = dict(zip(amt_df_grouped.index, amt_df_grouped.VAL_FINANCIAL_VALUE_12))
  with open('./dataset/Feature_Engineering_dict/R6M_CON_IMP_AMT_dict.pkl', 'wb') as f_amt :
    pickle.dump(amt_dict, f_amt)
  
  ##### 검사건수, 적발건수, 적발율 산출 #####

  # 위법물만 추출
  tmp_df = edf[edf.LABEL == 1]
  # CON_NAME_8로 groupby
  aggregations = {'CUS_REF_NO_7' : lambda x : x.mode(),
                  'LABEL' : 'count'}
  grouped_tmp_df = tmp_df.groupby(by='CON_NAME_8').agg(aggregations)

  # 산출 결과를 담을 DataFrame 생성
  ddf = pd.DataFrame()
  # 해당 업체
  ddf['CON_NAME_8'] = grouped_tmp_df.index
  # 업체에 해당하는 신고서 번호
  ddf['CUS_REF_NO_7'] = grouped_tmp_df.CUS_REF_NO_7.tolist()
  # 검사 건수
  ddf['CHECK_NUMB'] = grouped_tmp_df.LABEL.tolist()
  # 적발 건수 맵핑
  # 데이터 셋의 업체명과 적발 내역 데이터셋 안의 업체명은 일치하지 않음
  # 따라서 해당 업체의 신고서 번호를 기준으로 적발건수 추출 후 맵핑
  def get_numb_of_detected(x) :
    mask = ill_df['CUST_NUMBER'] == x['CUS_REF_NO_7']
    return int(ill_df[mask]['NUMB_OF_DETECTED'])
  ddf['NUMB_OF_DETECTED'] = ddf.apply(get_numb_of_detected, axis=1)
  # 적발율 산출 및 칼럼 생성
  ddf['DETECTED_RATIO'] = ddf.NUMB_OF_DETECTED / ddf.CHECK_NUMB

  # 해당 업체 이름 : 검사건수 dict 생성 / 저장
  check_numb_dict = dict(zip(ddf.CON_NAME_8, ddf.CHECK_NUMB))
  with open('./dataset/Feature_Engineering_dict/R6M_CON_NUMB_OF_INSPECTION_dict.pkl', 'wb') as f_in :
    pickle.dump(check_numb_dict, f_in)
  # 해당 업체 이름 : 적발건수 dict 생성 / 저장
  numb_of_detected_dict = dict(zip(ddf.CON_NAME_8, ddf.NUMB_OF_DETECTED))
  with open('./dataset/Feature_Engineering_dict/R6M_CON_NUMB_OF_DETECTED_dict.pkl', 'wb') as f_de :
    pickle.dump(numb_of_detected_dict, f_de)
  # 해당 업체 이름 : 적발률 dict 생성 / 저장
  detected_ratio_dict = dict(zip(ddf.CON_NAME_8, ddf.DETECTED_RATIO))
  with open('./dataset/Feature_Engineering_dict/R6M_CON_RATIO_OF_DETECTED_dict.pkl', 'wb') as f_ra :
    pickle.dump(detected_ratio_dict, f_ra)

  ##### 파생변수 맵핑 #####

  # 수입 건수
  df['R6M_CON_IMP_CNT'] = df.apply(lambda x : count_dict[x['CON_NAME_8']] if x['CON_NAME_8'] in count_dict.keys()
                                                                          else 0, axis=1)
  # 수입 금액
  df['R6M_CON_IMP_AMT'] = df.apply(lambda x : amt_dict[x['CON_NAME_8']] if x['CON_NAME_8'] in amt_dict.keys()
                                                                        else 0, axis=1)
  # 검사 건수
  df['R6M_CON_NUMB_OF_INSPECTION'] = df.apply(lambda x : check_numb_dict[x['CON_NAME_8']] if x['CON_NAME_8'] in check_numb_dict.keys()
                                                                                          else 0, axis=1)
  # 적발 건수
  df['R6M_CON_NUMB_OF_DETECTED'] = df.apply(lambda x : numb_of_detected_dict[x['CON_NAME_8']] if x['CON_NAME_8'] in numb_of_detected_dict.keys()
                                                                                              else 0, axis=1)
  # 적발율
  df['R6M_CON_RATIO_OF_DETECTED'] = df.apply(lambda x : detected_ratio_dict[x['CON_NAME_8']] if x['CON_NAME_8'] in detected_ratio_dict.keys()
                                                                                               else 0, axis=1)
  return df

mdf = get_dev_con(df)
dev_df1 = mdf[['R6M_CON_IMP_CNT', 'R6M_CON_IMP_AMT', 'R6M_CON_NUMB_OF_INSPECTION', 'R6M_CON_NUMB_OF_DETECTED', 'R6M_CON_RATIO_OF_DETECTED']] 
dev_df1

,R6M_CON_IMP_CNT,R6M_CON_IMP_AMT,R6M_CON_NUMB_OF_INSPECTION,R6M_CON_NUMB_OF_DETECTED,R6M_CON_RATIO_OF_DETECTED
0,3,800099.23,0,0,0.0
1,3,800099.23,0,0,0.0
2,3,800099.23,0,0,0.0
3,6,1395774.24,0,0,0.0
4,6,1395774.24,0,0,0.0
...,...,...,...,...,...
2146503,28,3697469.32,0,0,0.0
2146504,28,3697469.32,0,0,0.0
2146505,28,3697469.32,0,0,0.0
2146506,6,2306035.03,0,0,0.0


## 해당 HS CODE (HS_CODE 기준)

- 최근 6개월간 수입건수 : CUS_REF_NO_7 기준

- 최근 6개월간 수입금액 : COR_FINANCIAL_VALUE 기준

- 최근 6개월간 검사건수 : 2020. 05. 26 - 2020. 11. 25 까지 LABEL 1인 row

- 최근 6개월간 적발건수 : 적발내역 엑셀 참조 (해당 기간동안 2건)

- 최근 6개월간 적발율 : 적발건수 / 검사건수 = 모두 0이 나오므로 여기서는 제외

In [ ]:
# 적발건수 참조를 위해 ill_df중 2020. 05. 26 ~ 2020. 11. 25 에 해당하는 부분만 추출

tmp_df = ill_df.copy()

tmp_df['DATE'] = tmp_df.apply(lambda x : '20' + x['CUST_NUMBER'][11:13] +
                                         x['CUST_NUMBER'][9:11] + x['CUST_NUMBER'][7:9], axis=1)

tmp_df['DATE'] = tmp_df['DATE'].astype('datetime64')

mask = ('2020-05-26' <= tmp_df.DATE) & (tmp_df.DATE <= '2020-11-25')
ill_edf = tmp_df[mask]
ill_edf

,YEAR,IMP_COMPANY,NUMB_OF_DETECTED,CUSTOMS_CODE,CUST_NUMBER,HS_CODE,GOODS,INSPECTION_RESULT_CODE,VIOLATIONS,DATE
401,2020.0,ЧДММ Сафари,2,762249.0,762249/120620/0001093,3208209009,Краска,C204,Занижения таможенной стоимости,2020-06-12


In [ ]:
# HS CODE로 파생변수 만들 때 문제 발생

from datetime import datetime
from dateutil.relativedelta import relativedelta
import pickle

target_date = df['DATE'][2146407]
delta = target_date - relativedelta(months=6)
tdf = df[df.DATE >= delta]
tdf.reset_index(drop=True, inplace=True)

edf = tdf[['COM_COMBINED_NOMENCLATURE_33', 'CUS_REF_NO_7', 'COR_FINANCIAL_VALUE', 'LABEL']]

tmp_df = edf[edf.LABEL == 1]

aggregations = {'CUS_REF_NO_7' : lambda x : x.mode(),
                'LABEL' : 'count'}
grouped_tmp_df = tmp_df.groupby(by='COM_COMBINED_NOMENCLATURE_33').agg(aggregations)
grouped_tmp_df

# 아래에서 볼 수 있듯이 적발 HS CODE인 3208209009가 최근 6개월 추출한 데이터 셋에는 존재하지 않음..

,CUS_REF_NO_7,LABEL
COM_COMBINED_NOMENCLATURE_33,,
2715000000,762249/120620/0001093,2
3204170000,762249/120620/0001093,3
3206110000,762249/120620/0001093,2
3206498000,762249/120620/0001093,2
3209100009,762249/120620/0001093,2
3209900000,762249/120620/0001093,2
3214101009,762249/120620/0001093,2
3214109000,762249/120620/0001093,2


In [ ]:
# 하지만 본 데이터 셋 df에는 존재. 따라서 HS CODE 3208209009 에 해당하는 sample은 적발 건수 2를 바로 대입
# 단 적발률은 산출 불가능하므로 적발률 칼럼은 HS CODE에선 제외 
mask = df.COM_COMBINED_NOMENCLATURE_33 == 3208209009
df[mask].head(5)

,DATE,TRD_NAME_2,TRD_COUNTRY_2,TRD_ADDR_2,CUS_TOTAL_NUMBER_OF_ITEMS_5,CUS_TOTAL_NUMBER_OF_PACKAGES_6,CUS_REF_NO_7,CON_TIN_8,CON_NAME_8,CON_COUNTRY_8,CON_ADDR_8,IMP_TRADING_COUNTRY_11,VAL_FINANCIAL_VALUE_12,VAL_CURRENCY_12,IMP_CNT_OF_DISPATCH_EXP_CD_15,IMP_COUNTRY_OF_ORIGIN_16,DEL_DELIVERY_TERM_CODE_20,DEL_PLACE_OF_DELIVERY_20,TOT_CURRENCY_22,TOT_FINANCIAL_VALUE_22,IMP_EXCHANGE_RATE_23,IMP_INLAND_TRANSPORT_MODE_25,IMP_TRANSPORT_MODE_AT_BODR_26,OFF_CODE_30,GDS_GOODS_DESCRIPTION_31,GDS_ITEM_NUMBER_32,COM_COMBINED_NOMENCLATURE_33,IDG_COUNTRY_OF_ORIGIN_34,GDS_GROSS_MASS_35,PRF_PREFERENCE_CODE_1,PRF_PREFERENCE_CODE_2,PRF_PREFERENCE_CODE_3,PRF_PREFERENCE_CODE_4,IDG_PROCEDURE_REQUESTED_37,IDG_PREVIOUS_PROCEDURE_37,IDG_ADD_NATIONAL_PROC_37,IDG_NET_MASS_38,FIN_FINANCIAL_VALUE_42,COR_FINANCIAL_VALUE,COR_CURRENCY,STC_FINANCIAL_VALUE_46,CAL_TYPE_OF_TAX_47,CAL_TAX_BASE_47,CAL_RATE_OF_TAX_47,CAL_ADDITIONAL_RATE_OF_TAX_47,PAM_FINANCIAL_VALUE_47,CAL_METHOD_OF_PAYMENT_47,IMP_DATE_OF_DECLARATION_54,REP_TIN_54,PERSON_NAME_54,GEND_REFERENCE_54,GEND_ISSUE_DATE_54,ACCEPTANCE_DATE,LABEL,R6M_CON_IMP_CNT,R6M_CON_IMP_AMT,R6M_CON_NUMB_OF_INSPECTION,R6M_CON_NUMB_OF_DETECTED,R6M_CON_RATIO_OF_DETECTED
139,2017-04-13,"ООО ""ТЕХНОКОЛОР"", МОСКОВСКАЯ ОБЛАСТЬ",RU,ЩЕЛКОВО СВИРСКАЯ 14,3,135.0,762250/130417/0000132,510034385,"ООО ""КФ ФОРТУНА-КО""",TJ,"ХУДЖАНД СЭЗ ""СУГД""",RU,76431.43,TJS,RU,RU,FCA,ЩЕЛКОВО,RUB,464077.85,0.1539,30,30,762250,"РАСТВОРЫ ЛАКОВ, ИЗГОТОВЛЕННЫЕ НА ОСНОВЕ АКРИЛО...",3,3208209009,RU,103.0,О,А,А,А,78,0,44,100.0,20790.0,3354.14,TJS,395.0,20.0,3354.14,5.0,AD,167.71,УН,20170413,510025394,САТТОРОВ ФОТЕХХОН АБДУЛЛАЕВИЧ,СГ/01/0087,20160329.0,"13.04.17 11:02:10,905000000",0,0,0.00,0,0,0.0
140,2017-04-13,"ООО ""ТЕХНОКОЛОР"", МОСКОВСКАЯ ОБЛАСТЬ",RU,ЩЕЛКОВО СВИРСКАЯ 14,3,135.0,762250/130417/0000132,510034385,"ООО ""КФ ФОРТУНА-КО""",TJ,"ХУДЖАНД СЭЗ ""СУГД""",RU,76431.43,TJS,RU,RU,FCA,ЩЕЛКОВО,RUB,464077.85,0.1539,30,30,762250,"РАСТВОРЫ ЛАКОВ, ИЗГОТОВЛЕННЫЕ НА ОСНОВЕ АКРИЛО...",3,3208209009,RU,103.0,О,А,А,А,78,0,44,100.0,20790.0,3354.14,TJS,395.0,30.0,3521.85,18.0,AD,633.93,УН,20170413,510025394,САТТОРОВ ФОТЕХХОН АБДУЛЛАЕВИЧ,СГ/01/0087,20160329.0,"13.04.17 11:02:10,905000000",0,0,0.00,0,0,0.0
5049,2017-01-31,"СИНЬ-ЦЗЯНСКАЯ ТОРГОВАЯ КОМПАНИЯ ""ШИН-ХИ""",CN,УРУМЧИ,52,14902.0,762249/310117/0000153,610007405,"ООО ""А-ХАЕМ""",TJ,"РАСУЛОВСКИЙ Р/Н,ДЖ.ГУЛХОНА РЫНОК АШРОФ 57",CN,3404651.42,TJS,CN,CN,CIP,ХУДЖАНД,USD,430854.00,7.9021,30,30,762249,"ПРОЧИЕ РАСТВОРЫ КРАСОК И ЛАКОВ, ИЗГОТОВЛЕННЫХ ...",12,3208209009,CN,19812.0,О,О,О,О,40,0,0,19371.0,21041.0,166268.09,TJS,21041.0,20.0,166268.09,5.0,AD,8313.40,КТ,20170131,510022900,СУФИЕВ У,0043746,20090103.0,"31.01.17 09:11:56,124000000",0,0,0.00,0,0,0.0
5050,2017-01-31,"СИНЬ-ЦЗЯНСКАЯ ТОРГОВАЯ КОМПАНИЯ ""ШИН-ХИ""",CN,УРУМЧИ,52,14902.0,762249/310117/0000153,610007405,"ООО ""А-ХАЕМ""",TJ,"РАСУЛОВСКИЙ Р/Н,ДЖ.ГУЛХОНА РЫНОК АШРОФ 57",CN,3404651.42,TJS,CN,CN,CIP,ХУДЖАНД,USD,430854.00,7.9021,30,30,762249,"ПРОЧИЕ РАСТВОРЫ КРАСОК И ЛАКОВ, ИЗГОТОВЛЕННЫХ ...",12,3208209009,CN,19812.0,О,О,О,О,40,0,0,19371.0,21041.0,166268.09,TJS,21041.0,30.0,174581.49,18.0,AD,31424.67,КТ,20170131,510022900,СУФИЕВ У,0043746,20090103.0,"31.01.17 09:11:56,124000000",0,0,0.00,0,0,0.0
5119,2017-01-23,ООО ВЕГА СТИВИДОРС,LV,"РИГА, ЭЗЕРА 22",6,2271.0,762235/230117/0000088,610007790,ООО ИСТИКЛОЛ ВА КО,TJ,"МЕХРОБОД, НАВОИ 5",LV,420284.59,TJS,LV,DI,CPT,ДЖ. РАСУЛОВ,USD,53293.00,7.8863,20,20,762235,"КРАСКИ АВТОМОБИЛЬНЫЕ,1540 КОРОБОК.",6,3208209009,DE,35170.0,О,О,О,О,40,0,0,33880.0,37351.0,294561.19,TJS,37351.0,20.0,294561.19,5.0,AD,14728.06,БН,20170123,510013306,АВАЗОВ ДИЛШОДЖОН ГИЁСИДДИНОВИЧ,СГ00/036,20100621.0,"23.01.17 11:05:55,406000000",0,9,3750731.61,0,0,0.0


In [ ]:
# 아래서 볼 수 있듯이 COR_FINANCIAL_VALUE는 하나의 HS CODE - 하나의 신고서 - 하나의 품목 순서 당 하나씩 존재 
# 이를 파생변수 산출에 고려 

tmp_df = df[['CUS_REF_NO_7', 'COM_COMBINED_NOMENCLATURE_33', 'GDS_ITEM_NUMBER_32', 'COR_FINANCIAL_VALUE']]
aggregations = {'COR_FINANCIAL_VALUE' : lambda x : len(x.unique())}
tmp_df_grouped = tmp_df.groupby(['COM_COMBINED_NOMENCLATURE_33', 'CUS_REF_NO_7', 'GDS_ITEM_NUMBER_32']).agg(aggregations)

cnt = 0
for value in tmp_df_grouped.COR_FINANCIAL_VALUE :
  if value != 1. :
    cnt += 1
  else : pass

print('신고서 번호당 하나 이상의 COR_FINANCIAL_VALUE를 갖고 있는 경우 : {}'.format(cnt))

신고서 번호당 하나 이상의 COR_FINANCIAL_VALUE를 갖고 있는 경우 : 0


In [ ]:
# 파생변수 생성 사용자 함수 작성

def get_dev_code(df) :

  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  import pickle

  target_date = df['DATE'][2146407]
  delta = target_date - relativedelta(months=6)
  tdf = df[df.DATE >= delta]
  tdf.reset_index(drop=True, inplace=True)

  ##### 수입건수, 수입금액 산출 #####

  edf = tdf[['COM_COMBINED_NOMENCLATURE_33', 'CUS_REF_NO_7', 'GDS_ITEM_NUMBER_32', 'COR_FINANCIAL_VALUE', 'LABEL']]

  # 수입건수는 CUS_REF_NO_7의 유니크값 수
  aggregations1 = {'CUS_REF_NO_7' : lambda x : len(x.unique())}
  edf_grouped1 = edf.groupby('COM_COMBINED_NOMENCLATURE_33').agg(aggregations1)
  # 해당 HS CODE : 수입 건수 dict 작성 / pickle로 저장
  count_dict = dict(zip(edf_grouped1.index, edf_grouped1.CUS_REF_NO_7))
  with open('./dataset/Feature_Engineering_dict/R6M_CODE_IMP_CNT_dict.pkl', 'wb') as f_cnt :
    pickle.dump(count_dict, f_cnt)
  
  # 수입 금액
  # 집계함수로 최빈값을 추출 ; HS CODE - 신고서  - 물품 순서 당 COR_FINANCIAL_VALUE 는 한개이므로 
  aggregations2 = {'COR_FINANCIAL_VALUE' : lambda x : x.mode()}
  edf_grouped2 = edf.groupby(['COM_COMBINED_NOMENCLATURE_33', 'CUS_REF_NO_7', 'GDS_ITEM_NUMBER_32']).agg(aggregations2)
  # 3중 인덱스이므로 0번째 값 (COM_COMBINED_NOMENCLATURE_33) 만 리스트에 담음
  idx_list = []
  for i in range(len(edf_grouped2.index)) :
    idx_list.append(edf_grouped2.index[i][0])
  # 3중 인덱스를 COM_COMBINED_NOMENCLATURE_33 만으로 바꾼 DataFrame 생성 
  amt_df = pd.DataFrame({'COM_COMBINED_NOMENCLATURE_33' : idx_list,
                         'COR_FINANCIAL_VALUE' : edf_grouped2.COR_FINANCIAL_VALUE.values})
  # 집계함수 산출
  aggregations3 = {'COR_FINANCIAL_VALUE' : 'sum'}
  amt_df_grouped = amt_df.groupby('COM_COMBINED_NOMENCLATURE_33').agg(aggregations3)
  # 해당 HS CODE : 수입 금액 dict 작성 / pickle로 저장
  amt_dict = dict(zip(amt_df_grouped.index, amt_df_grouped.COR_FINANCIAL_VALUE))
  with open('./dataset/Feature_Engineering_dict/R6M_CODE_IMP_AMT_dict.pkl', 'wb') as f_amt :
    pickle.dump(amt_dict, f_amt)
  
  ##### 검사건수, 적발건수, 적발율 산출 #####

  # 위법물만 추출
  tmp_df = edf[edf.LABEL == 1]
  # COM_COMBINED_NOMENCLATURE_33 으로 groupby
  aggregations4 = {'LABEL' : 'count'}
  tmp_df_grouped = tmp_df.groupby(by='COM_COMBINED_NOMENCLATURE_33').agg(aggregations4)
  # 해당 HS CODE : 검사건수 dict 작성 / pickle로 저장
  check_numb_dict = dict(zip(tmp_df_grouped.index, tmp_df_grouped.LABEL))
  with open('./dataset/Feature_Engineering_dict/R6M_CODE_NUMB_OF_INSPECTION_dict.pkl', 'wb') as f_in :
    pickle.dump(check_numb_dict, f_in)
  # 해당 HS CODE : 적발건수 dict 작성 / pickle로 저장
  numb_of_detected_dict = {3208209009 : 2}
  with open('./dataset/Feature_Engineering_dict/R6M_CODE_NUMB_OF_DETECTED_dict.pkl', 'wb') as f_de :
    pickle.dump(numb_of_detected_dict, f_de)
  # 최근 6개월간 적발률은 3208209009가 최근 6개월 데이터셋에 존재하지 않아 모든 값이 0이 나오므로 생성하지 않음
  
  ##### 파생변수 맵핑 #####

  # 수입 건수
  df['R6M_CODE_IMP_CNT'] = df.apply(lambda x : count_dict[x['COM_COMBINED_NOMENCLATURE_33']] if x['COM_COMBINED_NOMENCLATURE_33'] in count_dict.keys()
                                                                          else 0, axis=1)
  # 수입 금액
  df['R6M_CODE_IMP_AMT'] = df.apply(lambda x : amt_dict[x['COM_COMBINED_NOMENCLATURE_33']] if x['COM_COMBINED_NOMENCLATURE_33'] in amt_dict.keys()
                                                                        else 0, axis=1)
  # 검사 건수
  df['R6M_CODE_NUMB_OF_INSPECTION'] = df.apply(lambda x : check_numb_dict[x['COM_COMBINED_NOMENCLATURE_33']] if x['COM_COMBINED_NOMENCLATURE_33'] in check_numb_dict.keys()
                                                                                                            else 0, axis=1)
  # 적발 건수
  df['R6M_CODE_NUMB_OF_DETECTED'] = df.apply(lambda x : numb_of_detected_dict[x['COM_COMBINED_NOMENCLATURE_33']] if x['COM_COMBINED_NOMENCLATURE_33'] in numb_of_detected_dict.keys()
                                                                                                                else 0, axis=1)
  return df

mdf = get_dev_code(df)
dev_df2 = mdf[['R6M_CODE_IMP_CNT', 'R6M_CODE_IMP_AMT', 'R6M_CODE_NUMB_OF_INSPECTION', 'R6M_CODE_NUMB_OF_DETECTED']] 
dev_df2

,R6M_CODE_IMP_CNT,R6M_CODE_IMP_AMT,R6M_CODE_NUMB_OF_INSPECTION,R6M_CODE_NUMB_OF_DETECTED
0,0,0.0,0,0
1,0,0.0,0,0
2,0,0.0,0,0
3,49,7322224.0,0,0
4,49,7322224.0,0,0
...,...,...,...,...
2146503,1715,74123910.4,0,0
2146504,1715,74123910.4,0,0
2146505,1715,74123910.4,0,0
2146506,70,1672623.4,0,0


## 해당 원산지 (IDG_COUNTRY_OF_ORIGIN_34 기준)

 - 최근 6개월간 수입건수 : CUS_REF_NO_7 기준

 - 최근 6개월간 수입금액 : COR_FINANCIAL_VALUE 기준

 - 최근 6개월간 검사건수 : 2020. 5. 26 - 2020. 11. 25 까지 LABEL 1인 row

 - 최근 6개월간 적발건수 : 적발내역 엑셀 참조 (해당 기간동안 2건)

 - 최근 6개월간 적발율 : 적발건수 / 검사건수

In [ ]:
# 파생변수 생성 사용자 함수 작성

def get_dev_country(df) :

  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  import pickle

  target_date = df['DATE'][2146407]
  delta = target_date - relativedelta(months=6)
  tdf = df[df.DATE >= delta]
  tdf.reset_index(drop=True, inplace=True)

  ##### 수입건수, 수입금액 산출 #####

  edf = tdf[['IDG_COUNTRY_OF_ORIGIN_34', 'CUS_REF_NO_7', 'COM_COMBINED_NOMENCLATURE_33', 'GDS_ITEM_NUMBER_32', 'COR_FINANCIAL_VALUE', 'LABEL']]

  # 수입건수는 CUS_REF_NO_7의 유니크값 수
  aggregations1 = {'CUS_REF_NO_7' : lambda x : len(x.unique())}
  edf_grouped1 = edf.groupby('IDG_COUNTRY_OF_ORIGIN_34').agg(aggregations1)
  # 해당 원산지 : 수입 건수 dict 작성 / pickle로 저장
  count_dict = dict(zip(edf_grouped1.index, edf_grouped1.CUS_REF_NO_7))
  with open('./dataset/Feature_Engineering_dict/R6M_COUNTRY_IMP_CNT_dict.pkl', 'wb') as f_cnt :
    pickle.dump(count_dict, f_cnt)
  
  # 수입 금액
  # 집계함수로 최빈값을 추출 ; 신고서 - HS CODE  - 물품 순서 당 COR_FINANCIAL_VALUE 는 한개이므로  
  aggregations2 = {'COR_FINANCIAL_VALUE' : lambda x : x.mode()}
  edf_grouped2 = edf.groupby(['IDG_COUNTRY_OF_ORIGIN_34', 'CUS_REF_NO_7', 'COM_COMBINED_NOMENCLATURE_33', 'GDS_ITEM_NUMBER_32']).agg(aggregations2)
  # 4중 인덱스이므로 0번째 값 (IDG_COUNTRY_OF_ORIGIN_34) 만 리스트에 담음
  idx_list = []
  for i in range(len(edf_grouped2.index)) :
    idx_list.append(edf_grouped2.index[i][0])
  # 4중 인덱스를 IDG_COUNTRY_OF_ORIGIN_34 만으로 바꾼 DataFrame 생성 
  amt_df = pd.DataFrame({'IDG_COUNTRY_OF_ORIGIN_34' : idx_list,
                         'COR_FINANCIAL_VALUE' : edf_grouped2.COR_FINANCIAL_VALUE.values})
  # 집계함수 산출
  aggregations3 = {'COR_FINANCIAL_VALUE' : 'sum'}
  amt_df_grouped = amt_df.groupby('IDG_COUNTRY_OF_ORIGIN_34').agg(aggregations3)
  # 해당 원산지 : 수입 금액 dict 작성 / pickle로 저장
  amt_dict = dict(zip(amt_df_grouped.index, amt_df_grouped.COR_FINANCIAL_VALUE))
  with open('./dataset/Feature_Engineering_dict/R6M_COUNTRY_IMP_AMT_dict.pkl', 'wb') as f_amt :
    pickle.dump(amt_dict, f_amt)
  
  ##### 검사건수, 적발건수, 적발율 산출 #####

  # 위법물만 추출
  tmp_df = edf[edf.LABEL == 1]
  # IDG_COUNTRY_OF_ORIGIN_34로 groupby
  aggregations4 = {'LABEL' : 'count'}
  grouped_tmp_df = tmp_df.groupby('IDG_COUNTRY_OF_ORIGIN_34').agg(aggregations4)
  # 산출 결과를 담을 DataFrame 생성
  ddf = pd.DataFrame()
  # 해당 원산지
  ddf['IDG_COUNTRY_OF_ORIGIN_34'] = grouped_tmp_df.index
  # 검사 건수
  ddf['CHECK_NUMB'] = grouped_tmp_df.LABEL.tolist()
  # 적발 건수
  ddf['NUMB_OF_DETECTED'] = 2
  # 적발율
  ddf['DETECTED_RATIO'] = ddf.NUMB_OF_DETECTED / ddf.CHECK_NUMB
  
  # 해당 원산지 : 검사건수 dict 생성 / 저장
  check_numb_dict = dict(zip(ddf.IDG_COUNTRY_OF_ORIGIN_34, ddf.CHECK_NUMB))
  with open('./dataset/Feature_Engineering_dict/R6M_COUNTRY_NUMB_OF_INSPECTION_dict.pkl', 'wb') as f_in :
    pickle.dump(check_numb_dict, f_in)
  # 해당 원산지 : 적발건수 dict 생성 / 저장
  numb_of_detected_dict = dict(zip(ddf.IDG_COUNTRY_OF_ORIGIN_34, ddf.NUMB_OF_DETECTED))
  with open('./dataset/Feature_Engineering_dict/R6M_COUNTRY_NUMB_OF_DETECTED_dict.pkl', 'wb') as f_de :
    pickle.dump(numb_of_detected_dict, f_de)
  # 해당 원산지 : 적발률 dict 생성 / 저장
  detected_ratio_dict = dict(zip(ddf.IDG_COUNTRY_OF_ORIGIN_34, ddf.DETECTED_RATIO))
  with open('./dataset/Feature_Engineering_dict/R6M_COUNTRY_RATIO_OF_DETECTED_dict.pkl', 'wb') as f_ra :
    pickle.dump(detected_ratio_dict, f_ra)

  ##### 파생변수 맵핑 #####

  # 수입 건수
  df['R6M_COUNTRY_IMP_CNT'] = df.apply(lambda x : count_dict[x['IDG_COUNTRY_OF_ORIGIN_34']] if x['IDG_COUNTRY_OF_ORIGIN_34'] in count_dict.keys()
                                                                                            else 0, axis=1)
  # 수입 금액
  df['R6M_COUNTRY_IMP_AMT'] = df.apply(lambda x : amt_dict[x['IDG_COUNTRY_OF_ORIGIN_34']] if x['IDG_COUNTRY_OF_ORIGIN_34'] in amt_dict.keys()
                                                                                          else 0, axis=1)
  # 검사 건수
  df['R6M_COUNTRY_NUMB_OF_INSPECTION'] = df.apply(lambda x : check_numb_dict[x['IDG_COUNTRY_OF_ORIGIN_34']] if x['IDG_COUNTRY_OF_ORIGIN_34'] in check_numb_dict.keys()
                                                                                                            else 0, axis=1)
  # 적발 건수
  df['R6M_COUNTRY_NUMB_OF_DETECTED'] = df.apply(lambda x : numb_of_detected_dict[x['IDG_COUNTRY_OF_ORIGIN_34']] if x['IDG_COUNTRY_OF_ORIGIN_34'] in numb_of_detected_dict.keys()
                                                                                                                else 0, axis=1)
  # 적발율
  df['R6M_COUNTRY_RATIO_OF_DETECTED'] = df.apply(lambda x : detected_ratio_dict[x['IDG_COUNTRY_OF_ORIGIN_34']] if x['IDG_COUNTRY_OF_ORIGIN_34'] in detected_ratio_dict.keys()
                                                                                                               else 0, axis=1)
  return df

mdf = get_dev_country(df)
dev_df3 = mdf[['R6M_COUNTRY_IMP_CNT', 'R6M_COUNTRY_IMP_AMT', 'R6M_COUNTRY_NUMB_OF_INSPECTION', 'R6M_COUNTRY_NUMB_OF_DETECTED', 'R6M_COUNTRY_RATIO_OF_DETECTED']] 
dev_df3

,R6M_COUNTRY_IMP_CNT,R6M_COUNTRY_IMP_AMT,R6M_COUNTRY_NUMB_OF_INSPECTION,R6M_COUNTRY_NUMB_OF_DETECTED,R6M_COUNTRY_RATIO_OF_DETECTED
0,9360,5.275566e+09,0,0,0.0
1,9360,5.275566e+09,0,0,0.0
2,9360,5.275566e+09,0,0,0.0
3,72,6.842362e+06,0,0,0.0
4,72,6.842362e+06,0,0,0.0
...,...,...,...,...,...
2146503,3264,2.467364e+08,0,0,0.0
2146504,3264,2.467364e+08,0,0,0.0
2146505,3264,2.467364e+08,0,0,0.0
2146506,4021,2.561359e+09,0,0,0.0


## 해당 관세사 (REP_TIN_54 기준)

 - 최근 6개월간 수입건수 : CUS_REF_NO_7 기준

 - 최근 6개월간 수입금액 : COR_FINANCIAL_VALUE 기준

 - 최근 6개월간 검사건수 : 2020. 5. 26 - 2020. 11. 25 까지 LABEL 1인 row

 - 최근 6개월간 적발건수 : 적발내역 엑셀 참조 (해당 기간동안 2건)

 - 최근 6개월간 적발율 : 적발건수 / 검사건수

In [ ]:
# 파생변수 생성 사용자 함수 작성

def get_dev_rep(df) :

  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  import pickle

  target_date = df['DATE'][2146407]
  delta = target_date - relativedelta(months=6)
  tdf = df[df.DATE >= delta]
  tdf.reset_index(drop=True, inplace=True)

  ##### 수입건수, 수입금액 산출 #####

  edf = tdf[['REP_TIN_54', 'CUS_REF_NO_7', 'COM_COMBINED_NOMENCLATURE_33', 'GDS_ITEM_NUMBER_32', 'COR_FINANCIAL_VALUE', 'LABEL']]

  # 수입건수는 CUS_REF_NO_7의 유니크값 수
  aggregations1 = {'CUS_REF_NO_7' : lambda x : len(x.unique())}
  edf_grouped1 = edf.groupby('REP_TIN_54').agg(aggregations1)
  # 해당 관세사 : 수입 건수 dict 작성 / pickle로 저장
  count_dict = dict(zip(edf_grouped1.index, edf_grouped1.CUS_REF_NO_7))
  with open('./dataset/Feature_Engineering_dict/R6M_REP_IMP_CNT_dict.pkl', 'wb') as f_cnt :
    pickle.dump(count_dict, f_cnt)
  
  # 수입 금액
  # 집계함수로 최빈값을 추출 ; 신고서 - HS CODE  - 물품 순서 당 COR_FINANCIAL_VALUE 는 한개이므로  
  aggregations2 = {'COR_FINANCIAL_VALUE' : lambda x : x.mode()}
  edf_grouped2 = edf.groupby(['REP_TIN_54', 'CUS_REF_NO_7', 'COM_COMBINED_NOMENCLATURE_33', 'GDS_ITEM_NUMBER_32']).agg(aggregations2)
  # 4중 인덱스이므로 0번째 값 (REP_TIN_54) 만 리스트에 담음
  idx_list = []
  for i in range(len(edf_grouped2.index)) :
    idx_list.append(edf_grouped2.index[i][0])
  # 4중 인덱스를 REP_TIN_54 만으로 바꾼 DataFrame 생성 
  amt_df = pd.DataFrame({'REP_TIN_54' : idx_list,
                         'COR_FINANCIAL_VALUE' : edf_grouped2.COR_FINANCIAL_VALUE.values})
  # 집계함수 산출
  aggregations3 = {'COR_FINANCIAL_VALUE' : 'sum'}
  amt_df_grouped = amt_df.groupby('REP_TIN_54').agg(aggregations3)
  # 해당 관세사 : 수입 금액 dict 작성 / pickle로 저장
  amt_dict = dict(zip(amt_df_grouped.index, amt_df_grouped.COR_FINANCIAL_VALUE))
  with open('./dataset/Feature_Engineering_dict/R6M_REP_IMP_AMT_dict.pkl', 'wb') as f_amt :
    pickle.dump(amt_dict, f_amt)
  
  ##### 검사건수, 적발건수, 적발율 산출 #####

  # 위법물만 추출
  tmp_df = edf[edf.LABEL == 1]
  # REP_TIN_54로 groupby
  aggregations4 = {'LABEL' : 'count'}
  grouped_tmp_df = tmp_df.groupby('REP_TIN_54').agg(aggregations4)
  # 산출 결과를 담을 DataFrame 생성
  ddf = pd.DataFrame()
  # 해당 원산지
  ddf['REP_TIN_54'] = grouped_tmp_df.index
  # 검사 건수
  ddf['CHECK_NUMB'] = grouped_tmp_df.LABEL.tolist()
  # 적발 건수
  ddf['NUMB_OF_DETECTED'] = 2
  # 적발율
  ddf['DETECTED_RATIO'] = ddf.NUMB_OF_DETECTED / ddf.CHECK_NUMB
  
  # 해당 원산지 : 검사건수 dict 생성 / 저장
  check_numb_dict = dict(zip(ddf.REP_TIN_54, ddf.CHECK_NUMB))
  with open('./dataset/Feature_Engineering_dict/R6M_REP_NUMB_OF_INSPECTION_dict.pkl', 'wb') as f_in :
    pickle.dump(check_numb_dict, f_in)
  # 해당 원산지 : 적발건수 dict 생성 / 저장
  numb_of_detected_dict = dict(zip(ddf.REP_TIN_54, ddf.NUMB_OF_DETECTED))
  with open('./dataset/Feature_Engineering_dict/R6M_REP_NUMB_OF_DETECTED_dict.pkl', 'wb') as f_de :
    pickle.dump(numb_of_detected_dict, f_de)
  # 해당 원산지 : 적발률 dict 생성 / 저장
  detected_ratio_dict = dict(zip(ddf.REP_TIN_54, ddf.DETECTED_RATIO))
  with open('./dataset/Feature_Engineering_dict/R6M_REP_RATIO_OF_DETECTED_dict.pkl', 'wb') as f_ra :
    pickle.dump(detected_ratio_dict, f_ra)

  ##### 파생변수 맵핑 #####

  # 수입 건수
  df['R6M_REP_IMP_CNT'] = df.apply(lambda x : count_dict[x['REP_TIN_54']] if x['REP_TIN_54'] in count_dict.keys()
                                                                              else 0, axis=1)
  # 수입 금액
  df['R6M_REP_IMP_AMT'] = df.apply(lambda x : amt_dict[x['REP_TIN_54']] if x['REP_TIN_54'] in amt_dict.keys()
                                                                            else 0, axis=1)
  # 검사 건수
  df['R6M_REP_NUMB_OF_INSPECTION'] = df.apply(lambda x : check_numb_dict[x['REP_TIN_54']] if x['REP_TIN_54'] in check_numb_dict.keys()
                                                                                              else 0, axis=1)
  # 적발 건수
  df['R6M_REP_NUMB_OF_DETECTED'] = df.apply(lambda x : numb_of_detected_dict[x['REP_TIN_54']] if x['REP_TIN_54'] in numb_of_detected_dict.keys()
                                                                                                  else 0, axis=1)
  # 적발율
  df['R6M_REP_RATIO_OF_DETECTED'] = df.apply(lambda x : detected_ratio_dict[x['REP_TIN_54']] if x['REP_TIN_54'] in detected_ratio_dict.keys()
                                                                                                 else 0, axis=1)
  return df

mdf = get_dev_rep(df)
dev_df4 = mdf[['R6M_REP_IMP_CNT', 'R6M_REP_IMP_AMT', 'R6M_REP_NUMB_OF_INSPECTION', 'R6M_REP_NUMB_OF_DETECTED', 'R6M_REP_RATIO_OF_DETECTED']] 
dev_df4

,R6M_REP_IMP_CNT,R6M_REP_IMP_AMT,R6M_REP_NUMB_OF_INSPECTION,R6M_REP_NUMB_OF_DETECTED,R6M_REP_RATIO_OF_DETECTED
0,2695,1.915663e+09,0,0,0.0
1,2695,1.915663e+09,0,0,0.0
2,2695,1.915663e+09,0,0,0.0
3,2695,1.915663e+09,0,0,0.0
4,2695,1.915663e+09,0,0,0.0
...,...,...,...,...,...
2146503,1508,3.124966e+08,0,0,0.0
2146504,1508,3.124966e+08,0,0,0.0
2146505,1508,3.124966e+08,0,0,0.0
2146506,1558,1.297993e+09,0,0,0.0


## Concat DataFrame

In [ ]:
dev_df = pd.concat([dev_df1, dev_df2, dev_df3, dev_df4], axis=1)
dev_df

,R6M_CON_IMP_CNT,R6M_CON_IMP_AMT,R6M_CON_NUMB_OF_INSPECTION,R6M_CON_NUMB_OF_DETECTED,R6M_CON_RATIO_OF_DETECTED,R6M_CODE_IMP_CNT,R6M_CODE_IMP_AMT,R6M_CODE_NUMB_OF_INSPECTION,R6M_CODE_NUMB_OF_DETECTED,R6M_COUNTRY_IMP_CNT,R6M_COUNTRY_IMP_AMT,R6M_COUNTRY_NUMB_OF_INSPECTION,R6M_COUNTRY_NUMB_OF_DETECTED,R6M_COUNTRY_RATIO_OF_DETECTED,R6M_REP_IMP_CNT,R6M_REP_IMP_AMT,R6M_REP_NUMB_OF_INSPECTION,R6M_REP_NUMB_OF_DETECTED,R6M_REP_RATIO_OF_DETECTED
0,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0
1,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0
2,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0
3,6,1395774.24,0,0,0.0,49,7322224.0,0,0,72,6.842362e+06,0,0,0.0,2695,1.915663e+09,0,0,0.0
4,6,1395774.24,0,0,0.0,49,7322224.0,0,0,72,6.842362e+06,0,0,0.0,2695,1.915663e+09,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146503,28,3697469.32,0,0,0.0,1715,74123910.4,0,0,3264,2.467364e+08,0,0,0.0,1508,3.124966e+08,0,0,0.0
2146504,28,3697469.32,0,0,0.0,1715,74123910.4,0,0,3264,2.467364e+08,0,0,0.0,1508,3.124966e+08,0,0,0.0
2146505,28,3697469.32,0,0,0.0,1715,74123910.4,0,0,3264,2.467364e+08,0,0,0.0,1508,3.124966e+08,0,0,0.0
2146506,6,2306035.03,0,0,0.0,70,1672623.4,0,0,4021,2.561359e+09,0,0,0.0,1558,1.297993e+09,0,0,0.0


# 시간 관련 파생함수

- LightGBM 및 Decision tree의 feature importances 와 tree plot 을 살펴보았을때 최상위에 위치하는 아래의 시간 관련 변수들에서 파생변수 추출
    - IMP_DATE_OF_DECLARATION_54
    - ACCEPTANCE_DATE
    - CUS_REF_NO_7
- LightGBM 알고리즘으로 학습/평가 하여 가장 성능이 좋은 조합을 선택
    - A_HOUR : ACCEPTANCE_DATE 에서 시간 추출
    - C_MONTH : CUS_REF_NO_7에서 월 추출


In [ ]:
# 시간 관련 함수들만 추출하여 DataFrame 생성
column_list = ['ACCEPTANCE_DATE', 'IMP_DATE_OF_DECLARATION_54', 'CUS_REF_NO_7']
edf = df[column_list]

# A_HOUR 칼럼 생성
edf['TIME'] = edf.apply(lambda x : x['ACCEPTANCE_DATE'][9:17], axis=1)
edf['D_TIME'] = pd.to_datetime(edf['TIME'])
edf['S_HOUR'] = edf['D_TIME'].dt.strftime('%H')
edf['A_HOUR'] = edf['S_HOUR'].astype(str)

# C_MONTH 칼럼 생성
edf['NEW_C_DATE'] = edf.apply(lambda x : '20' + x['CUS_REF_NO_7'][11:13] + x['CUS_REF_NO_7'][9:11] + x['CUS_REF_NO_7'][7:9], axis=1)
edf['C_MONTH'] = edf.apply(lambda x : x['NEW_C_DATE'][4:6], axis=1)
edf.C_MONTH = edf.C_MONTH.astype(str)

# 분석용 데이터 셋에도 C_MONTH와 A_HOUR 추가
dev_df['C_MONTH'] = edf.C_MONTH
dev_df['A_HOUR'] = edf.A_HOUR
dev_df['LABEL'] = df.LABEL

# 01 Basic + Date
dev_df.to_pickle('./dataset/41_Basic_Date.pkl')
dev_df

NameError: ignored

# 원본 데이터셋의 칼럼 삽입

- Feature Importnace 100 이상 칼럼 중 다른 파생변수 생성에 사용하지 않는 칼럼 2개를 변수로 삽입

    - CUS_TOTAL_NUMBER_OF_PACKAGES_6

    - IMP_EXCHANGE_RATE_23

In [ ]:
dev_df.insert(21, 'CUS_TOTAL_NUMBER_OF_PACKAGES_6', df['CUS_TOTAL_NUMBER_OF_PACKAGES_6'])
dev_df.insert(22, 'IMP_EXCHANGE_RATE_23', df['IMP_EXCHANGE_RATE_23'])

# 02 Basic + Date + Column2
dev_df.to_pickle('./dataset/42_Basic_Date_Column2.pkl')
dev_df

# 업체별 위법물 비율
- LightGBM 및 Decision tree의 feature importances 와 tree plot 을 살펴보았을때 상위에 위치하는 8번 항목 (CON_TIN_8, CON_NAME_8, CON_COUNTRY_8, CON_ADDR_8) 중 가장 영향력이 높은 CON_NAME_8을 활용 해 파생변수 생성



In [ ]:
##### 시간 엄청 오래 걸립니다 돌리지 마세요!!! #####
##### 아래에 있는 피클파일을 참조해 주세요 #####

# 사용할 변수들만 추출
edf = df[['CON_NAME_8', 'LABEL']]

# 위법물 비율을 담을 리스트 생성
result_list = []

# for문을 돌리기 위하여 CON_NAME_8의 유니크값만 추출
name_list = edf.CON_NAME_8.unique().tolist()

for value in name_list :
  # total = 해당 업체의 모든 샘플의 수
  mask = edf.CON_NAME_8 == value
  total = len(edf[mask])
  # illegal = 해당 업체의 모든 위법물 수 
  mask2 = (edf.CON_NAME_8 == value) & (edf.LABEL == 1)
  illegal = len(edf[mask2])
  # 위법물 비율
  result = illegal / total
  result_list.append(result)

# 해당 업체 : 위법물 비율 dictionary 피클로 저장
mapping_dict = dict(zip(name_list, result_list))
with open('./dataset/Feature_Engineering_dict/CON_ILLEGAL_RATIO_dict.pkl', 'wb') as f :
    pickle.dump(mapping_dict, f)

# 코드 돌리는데 너무 오래 걸리니 일단 결과값 나온거 저장
edf['CON_ILLEGAL_RATIO'] = edf.apply(lambda x : mapping_dict[x['CON_NAME_8']], axis=1)
edf.to_pickle('./dataset/CON_ILLEGAL_RATIO.pkl')
edf

,CON_NAME_8,LABEL,CON_ILLEGAL_RATIO
0,ООО ЧИТГАР,0,0.000000
1,ООО ЧИТГАР,0,0.000000
2,ООО ЧИТГАР,0,0.000000
3,"ООО ""РОСТОИ СУГД""",0,0.000000
4,"ООО ""РОСТОИ СУГД""",0,0.000000
...,...,...,...
2146503,"ЧДММ ""ОРЗУИ КОРВОН""",0,0.002103
2146504,"ЧДММ ""ОРЗУИ КОРВОН""",0,0.002103
2146505,"ЧДММ ""ОРЗУИ КОРВОН""",0,0.002103
2146506,ООО МАДИНА-2015,0,0.000000


In [ ]:
# 저장된 pkl 로드
illegal_ratio_df = pd.read_pickle('./dataset/CON_ILLEGAL_RATIO.pkl')
# 분석용 데이터셋에 파생변수 칼럼 삽입
dev_df['CON_ILLEGAL_RATIO'] = illegal_ratio_df.CON_ILLEGAL_RATIO

# 03 Basic + Date + Column2 + Illegal
dev_df.to_pickle('./dataset/43_Basic_Date_Column2_Illegal.pkl')

,R6M_CON_IMP_CNT,R6M_CON_IMP_AMT,R6M_CON_NUMB_OF_INSPECTION,R6M_CON_NUMB_OF_DETECTED,R6M_CON_RATIO_OF_DETECTED,R6M_CODE_IMP_CNT,R6M_CODE_IMP_AMT,R6M_CODE_NUMB_OF_INSPECTION,R6M_CODE_NUMB_OF_DETECTED,R6M_COUNTRY_IMP_CNT,R6M_COUNTRY_IMP_AMT,R6M_COUNTRY_NUMB_OF_INSPECTION,R6M_COUNTRY_NUMB_OF_DETECTED,R6M_COUNTRY_RATIO_OF_DETECTED,R6M_REP_IMP_CNT,R6M_REP_IMP_AMT,R6M_REP_NUMB_OF_INSPECTION,R6M_REP_NUMB_OF_DETECTED,R6M_REP_RATIO_OF_DETECTED,C_MONTH,A_HOUR,CON_ILLEGAL_RATIO,LABEL
0,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0,02,11,0.000000,0
1,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0,02,11,0.000000,0
2,3,800099.23,0,0,0.0,0,0.0,0,0,9360,5.275566e+09,0,0,0.0,2695,1.915663e+09,0,0,0.0,02,11,0.000000,0
3,6,1395774.24,0,0,0.0,49,7322224.0,0,0,72,6.842362e+06,0,0,0.0,2695,1.915663e+09,0,0,0.0,02,15,0.000000,0
4,6,1395774.24,0,0,0.0,49,7322224.0,0,0,72,6.842362e+06,0,0,0.0,2695,1.915663e+09,0,0,0.0,02,15,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2146503,28,3697469.32,0,0,0.0,1715,74123910.4,0,0,3264,2.467364e+08,0,0,0.0,1508,3.124966e+08,0,0,0.0,11,11,0.002103,0
2146504,28,3697469.32,0,0,0.0,1715,74123910.4,0,0,3264,2.467364e+08,0,0,0.0,1508,3.124966e+08,0,0,0.0,11,11,0.002103,0
2146505,28,3697469.32,0,0,0.0,1715,74123910.4,0,0,3264,2.467364e+08,0,0,0.0,1508,3.124966e+08,0,0,0.0,11,11,0.002103,0
2146506,6,2306035.03,0,0,0.0,70,1672623.4,0,0,4021,2.561359e+09,0,0,0.0,1558,1.297993e+09,0,0,0.0,11,16,0.000000,0


In [ ]:
# 04 Basic + Date + Illegal
dev_df.drop(['CUS_TOTAL_NUMBER_OF_PACKAGES_6', 'IMP_EXCHANGE_RATE_23'], axis=1, inplace=True)
dev_df.to_pickle('./dataset/44_Basic_Date_Illegal.pkl')
dev_df

다른 파생변수도 생성하려 했으나 이미 precision 값이 1에 가까워 중단